In [28]:
# imports

import panel as pn
pn.extension('plotly')
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pprint as pp
import os
from pathlib import Path
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')
import fantasy_football as f_f
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

# Set the Mapbox API
px.set_mapbox_access_token("pk.eyJ1IjoiYW1hY2s1NiIsImEiOiJja3RldzZmcnAwMmd5Mm5wZngyejR2cm5iIn0.JohlA6vRxmH-iugH7u_xXA")

# Import all the CSVs from the Data of the last 10 years using custom fantasy_football.py library

full_data = f_f.read_fantasy_data(2010, 2020)

# Read in coordinate data

coor_path = Path('../Data/RawData/coordinates.csv')
coordinates = pd.read_csv(coor_path)

# Manipulate coordinate data

coordinates.drop(labels=['zip', 'pic', 'Unnamed: 6', 'Conference'], axis=1, inplace=True)
coordinates = coordinates.set_index('Team')

# Clean up the combined data

full_data = full_data[full_data['FantasyPoints'] != 0]
full_data = full_data[full_data['Tm'] != '2TM']
full_data = full_data[full_data['Tm'] != '3TM']
full_data = full_data[full_data['Tm'] != '4TM']
full_data = full_data.dropna()
full_data.to_csv('../Data/CleanData/full_data.csv', encoding='utf-8')

player_mean = full_data.groupby(['Player', 'Tm']).mean().sort_values(by='FantasyPoints', ascending=False, na_position='first')
player_mean.hvplot.bar(width=800, rot=90, groupby="Tm", title="Best Players By Team")

:DynamicMap   [Tm]
   :Bars   [Player,Tm]   (FantasyPoints)

In [29]:
# def lowest_players():
#     player_mean = full_data.groupby(['Player']).mean().sort_values(by='FantasyPoints', ascending=True, na_position='first').nsmallest(15,"FantasyPoints")
#     fig_lowest=go.Figure()
#     fig_lowest.add_traces(go.Bar(x=player_mean.index, y = player_mean.FantasyPoints,marker_color="#d10a1b"))
#     fig_lowest.update_layout(title_text='15 Players with Lowest FantasyPoints')
#     return fig_lowest

In [30]:
def points_team():
    fig_team = plt.figure()
    team_mean = full_data.groupby(['Tm']).mean().sort_values(by='FantasyPoints', ascending=False, na_position='first')
    #fig = team_mean.hvplot.bar(width=800, rot=90).opts(color="Tm")
    fig_team = px.bar(x=team_mean.index,y=team_mean.FantasyPoints,color=team_mean.index,
                 color_discrete_sequence=["#07518a"])
    fig_team.update_layout(title_text='Fantasy Points by Team')
    fig_team.update_layout(yaxis_title="Fantasy Points")
    fig_team.update_layout(xaxis_title="Team")
    fig_team.update_traces(hovertemplate='Team: %{x} <br>:FantasyPoints %{y}')
    fig_team.update_layout(showlegend=False) 
    return fig_team

In [31]:
team_divisions=[]
for each_team in full_data["Tm"]:
    if each_team=="NYG" or each_team=="WAS" or each_team=="PHI" or each_team=="DAL":
        team_divisions.append("NFC East")
    elif each_team=="LAR" or each_team=="SFO" or each_team=="SEA" or each_team=="ARI":
        team_divisions.append("NFC West")
    elif each_team=="CHI" or  each_team=="GNB" or each_team=="MIN" or each_team=="DET":
        team_divisions.append("NFC North")
    elif each_team=="NOR" or each_team=="ATL" or each_team=="TAM" or each_team=="CAR":
        team_divisions.append("NFC South")
    elif each_team=="NWE" or each_team=="BUF" or each_team=="NYJ" or each_team=="MIA":
        team_divisions.append("AFC East")
    elif each_team=="DEN" or each_team=="KAN" or each_team=="LAC" or each_team=="OAK":
        team_divisions.append("AFC West")
    elif each_team=="PIT" or each_team=="CIN" or each_team=="CLE" or each_team=="BAL":
        team_divisions.append("AFC North")
    elif each_team=="HOU" or each_team=="IND" or each_team=="TEN" or each_team=="JAX":
        team_divisions.append("AFC South")
    else:
        team_divisions.append(np.nan)
    

In [32]:
full_data_divisions=full_data.copy()
full_data_divisions["Divisions"]=team_divisions

In [33]:
full_data_divisions=full_data_divisions.groupby("Divisions").sum()
full_data_divisions.reset_index(inplace=True)
full_data_divisions=full_data_divisions.sort_values(by="FantasyPoints")

In [34]:
def points_division():
    fig_division = plt.figure()
    fig_division = px.bar(full_data_divisions, x='Divisions', y='FantasyPoints',color='Divisions')
    fig_division.update_layout(title_text='Fantasy Points by division')
    fig_division.update_layout(yaxis_title="Fantasy Points")
    return fig_division

In [35]:
def points_position():
    fig_position = plt.figure()
    position_mean = full_data.groupby(['Pos']).mean().sort_values(by='FantasyPoints', ascending=False, na_position='first')
    position_mean=position_mean.iloc[:-1,]
    fig_position = px.bar(x=position_mean.index,y=position_mean.FantasyPoints,color=position_mean.index,
                 color_discrete_sequence=px.colors.qualitative.G10)
    fig_position.update_layout(title_text='Fantasy Points by position')
    fig_position.update_layout(yaxis_title="Fantasy Points")
    fig_position.update_layout(xaxis_title="Position")
    fig_position.update_traces(hovertemplate='Position: %{x} <br>:FantasyPoints %{y}')
     
    return fig_position

In [36]:
def points_top_players():
    best_team = full_data.groupby(['Player']).mean().sort_values(by='FantasyPoints', ascending=False, na_position='first').nlargest(15, 'FantasyPoints')
    #fig = best_team.hvplot.bar(width=800, rot=90)
    fig=go.Figure()
    fig.add_traces(go.Bar(x=best_team.index, y = best_team.FantasyPoints,marker_color="#77913a"))
    fig.update_layout(title_text='Top 15 Players')
    return fig
    return fig

In [37]:
def team_mean_map():
    fig = plt.figure()
    team_mean = full_data.groupby(['Tm']).mean().sort_values(by='FantasyPoints', ascending=False, na_position='first')
    coords_and_mean = pd.concat([team_mean, coordinates], axis=1)
    coords_and_mean = coords_and_mean.reset_index()
    coords_and_mean = coords_and_mean.rename({'index': 'Team'}, axis=1)

    fig = px.scatter_mapbox(coords_and_mean, lat='latitude', lon='longitude', color="Team", size="FantasyPoints", zoom=2)
    return fig

In [38]:
def rushing_yards():
#     data_2010_yds = pd.read_csv(file_path_2010)
#     data_2010_yds['Year'] = '2010'
#     data_2011_yds = pd.read_csv(file_path_2011)
#     data_2011_yds['Year'] = '2011'
#     data_2012_yds = pd.read_csv(file_path_2012)
#     data_2012_yds['Year'] = '2012'
#     data_2013_yds = pd.read_csv(file_path_2013)
#     data_2013_yds['Year'] = '2013'
#     data_2014_yds = pd.read_csv(file_path_2014)
#     data_2014_yds['Year'] = '2014'
#     data_2015_yds = pd.read_csv(file_path_2015)
#     data_2015_yds['Year'] = '2015'
#     data_2016_yds = pd.read_csv(file_path_2016)
#     data_2016_yds['Year'] = '2016'
#     data_2017_yds = pd.read_csv(file_path_2017)
#     data_2017_yds['Year'] = '2017'
#     data_2018_yds = pd.read_csv(file_path_2018)
#     data_2018_yds['Year'] = '2018'
#     data_2019_yds = pd.read_csv(file_path_2019)
#     data_2019_yds['Year'] = '2019'
#     data_2010_yds = data_2010_yds[["Year","Player","Tm","Pos","RushingYds","FantasyPoints"]]
#     data_2011_yds = data_2011_yds[["Year","Player","Tm","Pos","RushingYds","FantasyPoints"]]
#     data_2012_yds = data_2012_yds[["Year","Player","Tm","Pos","RushingYds","FantasyPoints"]]
#     data_2013_yds = data_2013_yds[["Year","Player","Tm","Pos","RushingYds","FantasyPoints"]]
#     data_2014_yds = data_2014_yds[["Year","Player","Tm","Pos","RushingYds","FantasyPoints"]]
#     data_2015_yds = data_2015_yds[["Year","Player","Tm","Pos","RushingYds","FantasyPoints"]]
#     data_2016_yds = data_2016_yds[["Year","Player","Tm","Pos","RushingYds","FantasyPoints"]]
#     data_2017_yds = data_2017_yds[["Year","Player","Tm","Pos","RushingYds","FantasyPoints"]]
#     data_2018_yds = data_2018_yds[["Year","Player","Tm","Pos","RushingYds","FantasyPoints"]]
#     data_2019_yds = data_2019_yds[["Year","Player","Tm","Pos","RushingYds","FantasyPoints"]]
#     full_data_yds = pd.concat([data_2010_yds, data_2011_yds, data_2012_yds, data_2013_yds, data_2014_yds, 
#                            data_2015_yds, data_2016_yds, data_2017_yds, data_2018_yds, data_2019_yds])
    full_data_yds = f_f.read_rushing_yds(2010,2020)
    #print(full_data_yds.head())
    fig = plt.figure()
    full_data_yds = full_data_yds[full_data_yds['FantasyPoints'] != 0]
    full_data_yds = full_data_yds[full_data_yds['Tm'] != '2TM']
    full_data_yds = full_data_yds[full_data_yds['Tm'] != '3TM']
    full_data_yds = full_data_yds[full_data_yds['Tm'] != '4TM']
    fig = plt.figure()
    corr = np.corrcoef(full_data_yds["RushingYds"], full_data_yds["FantasyPoints"])
    fig = px.scatter(full_data_yds, x="RushingYds", y="FantasyPoints", trendline="ols", color='Year')
    return fig

In [39]:
#scatter_box = full_data_yds.hvplot.scatter(x='RushingYds', y='FantasyPoints', c='Year', groupby="Year")

In [40]:
# Create a Title for the Dashboard

title = pn.pane.Markdown("""
## Fantasy Football Project
""")

In [41]:
# Top Players by Position Graphs
def top_quarterbacks():
    fig_team = plt.figure()
    top_20_qbs = full_data[full_data["Pos"]=="QB"]
    top_20_qbs=top_20_qbs.groupby(['Player']).mean().sort_values(by='FantasyPoints', ascending=False, na_position='first').nlargest(20,'FantasyPoints')
    fig_team = px.bar(x=top_20_qbs.index,y=top_20_qbs.FantasyPoints,color=top_20_qbs.index,
                 color_discrete_sequence=["#de4a1d"])
    fig_team.update_layout(title_text='Top 20 Quarterbacks ')
    fig_team.update_layout(yaxis_title="Fantasy Points")
    fig_team.update_layout(xaxis_title="Player")
    fig_team.update_traces(hovertemplate='Player: %{x} <br>FantasyPoints: %{y}')
    fig_team.update_layout(showlegend=False) 
    return fig_team
    

In [42]:
def top_runningbacks():
    fig_team = plt.figure()
    top_20_rbs = full_data[full_data["Pos"]=="RB"]
    top_20_rbs=top_20_rbs.groupby(['Player']).mean().sort_values(by='FantasyPoints', ascending=False, na_position='first').nlargest(20,'FantasyPoints')
    fig_team = px.bar(x=top_20_rbs.index,y=top_20_rbs.FantasyPoints,color=top_20_rbs.index,
                 color_discrete_sequence=["#c53ac9"])
    fig_team.update_layout(title_text='Top 20 Running Backs ')
    fig_team.update_layout(yaxis_title="Fantasy Points")
    fig_team.update_layout(xaxis_title="Player")
    fig_team.update_traces(hovertemplate='Player: %{x} <br>FantasyPoints: %{y}')
    fig_team.update_layout(showlegend=False) 
    return fig_team
    

In [43]:
def top_wide_receivers():
    fig_team = plt.figure()
    top_20_wrs = full_data[full_data["Pos"]=="WR"]
    top_20_wrs=top_20_wrs.groupby(['Player']).mean().sort_values(by='FantasyPoints', ascending=False, na_position='first').nlargest(20,'FantasyPoints')
    fig_team = px.bar(x=top_20_wrs.index,y=top_20_wrs.FantasyPoints,color=top_20_wrs.index,
                 color_discrete_sequence=["#66d0d4"])
    fig_team.update_layout(title_text='Top 20 Wide Receivers ')
    fig_team.update_layout(yaxis_title="Fantasy Points")
    fig_team.update_layout(xaxis_title="Player")
    fig_team.update_traces(hovertemplate='Player: %{x} <br>FantasyPoints: %{y}')
    fig_team.update_layout(showlegend=False) 
    return fig_team
    

In [44]:
def top_tight_ends():
    fig_team = plt.figure()
    top_20_tes = full_data[full_data["Pos"]=="TE"]
    top_20_tes=top_20_tes.groupby(['Player']).mean().sort_values(by='FantasyPoints', ascending=False, na_position='first').nlargest(20,'FantasyPoints')
    fig_team = px.bar(x=top_20_tes.index,y=top_20_tes.FantasyPoints,color=top_20_tes.index,
                 color_discrete_sequence=["#169139"])
    fig_team.update_layout(title_text='Top 20 Tight Ends')
    fig_team.update_layout(yaxis_title="Fantasy Points")
    fig_team.update_layout(xaxis_title="Player")
    fig_team.update_traces(hovertemplate='Player: %{x} <br>FantasyPoints: %{y}')
    fig_team.update_layout(showlegend=False) 
    return fig_team
    

In [45]:
# Distribution Charts
def distribution_quarterbacks():
    fig = plt.figure
    qbs = full_data[full_data['Pos'] == 'QB'].groupby('Player').mean().nlargest(30, 'FantasyPoints')
    qbs_mean = int(np.mean(qbs))
    qbs_stdev = int(np.std(qbs))
    group_labels=["Quaterbacks Distribution"]
    #print(f"The mean for the top 30 QBs is: {qbs_mean} and the standard deviation is: {qbs_stdev}")
    fig = ff.create_distplot([qbs["FantasyPoints"]],group_labels,bin_size=.2, show_rug=False,show_hist=False)
    fig.update_layout(title_text='Top 30 Quaterbacks')
    fig.update_layout(showlegend=False) 
    
    return fig

In [46]:
def distribution_running_backs():
    fig = plt.figure
    rbs = full_data[full_data['Pos'] == 'RB'].groupby('Player').mean().nlargest(30, 'FantasyPoints')
    rbs_mean = int(np.mean(rbs))
    rbs_stdev = int(np.std(rbs))
    group_labels=["Running Backs Distribution"]
    #print(f"The mean for the top 30 RBs is: {rbs_mean} and the standard deviation is: {rbs_stdev}")
    fig = ff.create_distplot([rbs["FantasyPoints"]],group_labels,bin_size=.2, show_rug=False,show_hist=False)
    fig.update_layout(title_text='Top 30 Running Backs')
    fig.update_layout(showlegend=False) 
    
    return fig

In [47]:
def distribution_wide_receivers():
    fig = plt.figure
    wrs = full_data[full_data['Pos'] == 'WR'].groupby('Player').mean().nlargest(30, 'FantasyPoints')
    wrs_mean = int(np.mean(wrs))
    wrs_stdev = int(np.std(wrs))
    #print(f"The mean for the top 30 WRs is: {wrs_mean} and the standard deviation is: {wrs_stdev}")
    group_labels=["Wide Receivers Distribution"]
    fig = ff.create_distplot([wrs["FantasyPoints"]],group_labels,bin_size=.2, show_rug=False,show_hist=False)
    fig.update_layout(title_text='Top 30 Wide Receivers')
    fig.update_layout(showlegend=False) 
    
    return fig

In [48]:
def distribution_tight_ends():
    fig = plt.figure
    tes = full_data[full_data['Pos'] == 'TE'].groupby('Player').mean().nlargest(30, 'FantasyPoints')
    tes_mean = int(np.mean(tes))
    tes_stdev = int(np.std(tes))
    #print(f"The mean for the top 30 TEs is: {tes_mean} and the standard deviation is: {tes_stdev}")
    group_labels=["Tight Ends Distribution"]
    fig = ff.create_distplot([tes["FantasyPoints"]],group_labels,bin_size=.2, show_rug=False,show_hist=False)
    fig.update_layout(title_text='Top 30 Tight Ends')
    fig.update_layout(showlegend=False) 
    return fig

In [49]:
#rushing_yards()

In [50]:
# Create a tab layout for the dashboard

tabs = pn.Tabs(
    ("Players", pn.Column(points_top_players)), 
    ("Positions", pn.Column(points_position)),
    ("Teams", pn.Column(points_team)),
    ("Division",pn.Column(points_division)),
    ("Top Players Stats", pn.Column(top_quarterbacks, top_runningbacks, top_wide_receivers, top_tight_ends)),
    ("Top Player Distribution", pn.Column(distribution_quarterbacks, distribution_running_backs, distribution_wide_receivers, distribution_tight_ends)),
    ("Rushing Yards Scatter", pn.Column(rushing_yards)),
    ("Team Mean Map", pn.Column("Mean points of team fantasy points plotted on the map", team_mean_map))
)

In [51]:
# Create the dashboard

dashboard = pn.Column(title, tabs)

In [52]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 1500;

<IPython.core.display.Javascript object>

In [53]:
# Serve the# dashboard

#dashboard.servable()

In [54]:
dashboard.show()

Launching server at http://localhost:54829
